<a href="https://colab.research.google.com/github/Josephnyingi/Crop-Disease-Detection/blob/main/Crop_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

Plants are susceptible to diseases due to factors such as fertilizer use, farming practices, and environmental conditions, which negatively impact agricultural yields and the economy. Early detection of plant diseases is crucial for farmers to cultivate crops effectively and efficiently, both in terms of quality and quantity. Thus, disease detection in plants is a vital aspect of agriculture.

# Download dataset

Initially, we retrieve the PlantVillage dataset from Google Drive using its unique ID, then extract the contents of the downloaded PlantVillage.zip file into the designated PlantVillage dataset folder

# Import Libraries


In [1]:
#Library
import pandas as pd
import keras
import numpy as np
import pickle
import cv2
import os
import matplotlib.pyplot as plt
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
!pip install gdown
!apt-get update && apt-get install fil
!pip install keras_preprocessing


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [71.6 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:1

# Load the dataset

1. Mount your Google Drive:

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


Copied the dataset to Google Drive: Downloaded the dataset and saved it to Google Drive.
Then accessed the dataset in Colab 

In [3]:
import os
os.chdir('/content/gdrive/MyDrive/crowdai')

Initializing a few parameters required for the image dataset preprocessing.

In [4]:
# Dimension of resized image
DEFAULT_IMAGE_SIZE = tuple((256, 256))

# Number of images used to train the model
N_IMAGES = 100

# Path to the dataset folder
root_dir = '/content/gdrive/MyDrive/crowdai'

train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')

In [5]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, DEFAULT_IMAGE_SIZE)   
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer

# Set the path to the dataset directory
dataset_path = "/content/gdrive/MyDrive/crowdai"

# Get the list of image filenames and their corresponding labels
image_list = os.listdir(dataset_path)
label_list = [os.path.basename(os.path.dirname(os.path.join(dataset_path, image))) for image in image_list]

# Use LabelBinarizer to convert the string labels to binary labels
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)

# Create an empty list to store the preprocessed image data
preprocessed_images = []

# Loop through the image filenames
for image_filename in image_list:
    # Read the image using OpenCV
    image = cv2.imread(os.path.join(dataset_path, image_filename))

    # Check if the image was successfully read
    if image is not None:
        # Resize the image to a fixed size (e.g., 224x224)
        resized_image = cv2.resize(image, (224, 224))
        # Convert the image to a NumPy array and normalize the pixel values
        preprocessed_image = np.array(resized_image, dtype=np.float16) / 225.0
        # Add the preprocessed image to the list
        preprocessed_images.append(preprocessed_image)
    else:
        print(f"Failed to read image: {image_filename}")

# Convert the list of preprocessed images to a NumPy array
np_image_list = np.array(preprocessed_images)

# Check the number of images loaded for training
image_len = len(image_list)
print(f"Total number of images: {image_len}")

# Check the shape of the image data and label data
print(f"Image data shape: {np_image_list.shape}")
print(f"Label data shape: {image_labels.shape}")


Failed to read image: c_4
Failed to read image: c_2
Failed to read image: c_5
Failed to read image: c_21
Failed to read image: c_32


In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer

# Define the paths to your dataset
train_dir = '/content/gdrive/MyDrive/crowdai'
valid_dir = '/content/gdrive/MyDrive/crowdai'

# Define the image data generator with optional data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Use flow_from_directory to load the image data and labels
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Get the number of classes
num_classes = len(train_generator.class_indices)

# Align the image and label data into numpy arrays
train_samples = len(train_generator.filenames)
train_image_array = np.zeros((train_samples, 224, 224, 3), dtype=np.float32)
train_label_array = np.zeros((train_samples, num_classes), dtype=np.float32)

for i in range(0, train_samples, train_generator.batch_size):
    images, labels = train_generator.next()
    batch_size = images.shape[0]
    train_image_array[i:i+batch_size] = images
    train_label_array[i:i+batch_size] = labels

valid_samples = len(valid_generator.filenames)
valid_image_array = np.zeros((valid_samples, 224, 224, 3), dtype=np.float32)
valid_label_array = np.zeros((valid_samples, num_classes), dtype=np.float32)

for i in range(0, valid_samples, valid_generator.batch_size):
    images, labels = valid_generator.next()
    batch_size = images.shape[0]
    valid_image_array[i:i+batch_size] = images
    valid_label_array[i:i+batch_size] = labels
# Use LabelBinarizer to binarize the labels
label_binarizer = LabelBinarizer()
train_label_array = label_binarizer.fit_transform(np.argmax(train_label_array, axis=1))
valid_label_array = label_binarizer.transform(np.argmax(valid_label_array, axis=1))


Counting Total number of images 

In [ ]:
import os

# Define the path to the root directory of the Plant Village dataset
dataset_path = '/content/gdrive/MyDrive/crowdai'

# Initialize a count of the number of images
count = 0

# Loop through the subdirectories of the dataset path
for subdir, dirs, files in os.walk(dataset_path):
    # Loop through the files in the subdirectory
    for file in files:
        # Check if the file is an image file with a .jpg or .png extension
        if file.endswith('.jpg') or file.endswith('.png'):
            # Increment the count of images
            count += 1

# Print the count of images
print('Number of images:', count)


# Checking distribution of labels

Checked the distribution of labels in a dataset as it is an important step in preparing the data for machine learning. This can help you determine if the dataset is balanced or imbalanced, which can have an impact on the performance of your model.

To check the distribution of labels in the Plant Village dataset, you can use the following code:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Load the data
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

checking the distribution of labels in a dataset is an important step in preparing the data for machine learning. This can help you determine if the dataset is balanced or imbalanced, which can have an impact on the performance of your model.
The code uses the numpy library to count the number of occurrences of each label in the y_train array. The unique_labels variable contains the unique labels in the array, and the count_labels variable contains the number of occurrences of each label.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Count the number of occurrences of each label
unique_labels, count_labels = np.unique(y_train, return_counts=True)

# Plot the distribution of labels
plt.bar(unique_labels, count_labels)
plt.xlabel("Label")
plt.ylabel("Count")
plt.title("Distribution of Labels")
plt.show()


# Preprocessing data

1. Splitting the data into training and validation sets: use the train_test_split function from the scikit-learn library to split data into training and validation sets. This allos to evaluate model's performance on a validation set, which is unseen data.

# Total number of classes

To examine the labels/classes in the crowdia directory read the names of the subdirectories in the dataset directory. Each subdirectory corresponds to a different class or label in the dataset. Here is how i get the list of classes in the dataset

In [ ]:
import os

# Set the directory path to the Plant Village dataset
data_dir = '/content/gdrive/MyDrive/crowdai'

# Get the list of subdirectories in the dataset directory
classes = os.listdir(data_dir)

# Get the total number of classes
num_classes = len(classes)

# Print the total number of classes
print(f'The Plant Village dataset contains {num_classes} classes')



In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator


augment = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.2, 
                             zoom_range=0.2, horizontal_flip=True, 
                             fill_mode="nearest")

In [ ]:
print("[INFO] Splitting data to train and test...")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)
